Add your name and PID
<br> Kush Parmar A14032759<br> Alex Biz A14455796

## Introduction

### Overview

### Research Question

### Hypothesis

### Background

###  Dataset Description

## Data Cleaning and Pre-processing

### Setup

In [9]:
# import libraries
import pandas as pd
import spotipy
from profanity_check import predict, predict_prob
import numpy as np

In [8]:
# load in raw data
raw_data = pd.read_csv("./dataset/lyrics.csv")
raw_data.index = raw_data["index"]
raw_data = raw_data.drop(columns=["index"])

raw_data

,song,year,artist,genre,lyrics
index,,,,,
0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu..."
1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy,\nit's like you see..."
2,honesty,2009,beyonce-knowles,Pop,If you search\nFor tenderness\nIt isn't hard t...
3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote..."
4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."
5,all-i-could-do-was-cry,2009,beyonce-knowles,Pop,I heard\nChurch bells ringing\nI heard\nA choi...
6,once-in-a-lifetime,2009,beyonce-knowles,Pop,This is just another day that I would spend\nW...
7,waiting,2009,beyonce-knowles,Pop,"Waiting, waiting, waiting, waiting\nWaiting, w..."
8,slow-love,2009,beyonce-knowles,Pop,[Verse 1:]\nI read all of the magazines\nwhile...


In [12]:
# get popularity data from spotify API
spot = spotipy.Spotify()

# iterate over all songs 
for i in range(len(raw_data)):
    song = raw_data.iloc[i]["song"]
    artist = raw_data.iloc[i]["artist"]
    
    # replace hyphen with spaces
    song = song.replace("-", " ")
    artist = artist.replace("-", " ")
    
    if i > 10:
        break

ego remix
beyonce knowles
then tell me
beyonce knowles
honesty
beyonce knowles
you are my rock
beyonce knowles
black culture
beyonce knowles
all i could do was cry
beyonce knowles
once in a lifetime
beyonce knowles
waiting
beyonce knowles
slow love
beyonce knowles
why don t you love me
beyonce knowles
save the hero
beyonce knowles
telephone
beyonce knowles


### Cleaning

In [ ]:
# Drop NaN
# Drop non English versions, and remixes

In [2]:
# clean lyrics - replace \n with " "

In [ ]:
# add profanity column

## Data Analysis

In [1]:
# summary, and description of data

In [ ]:
# EDA